# Lab | LangChain Med

## Objectives

- continue on with lesson 2' example, use different datasets to test what we did in class. Some datasets are suggested in the notebook but feel free to scout other datasets on HuggingFace or Kaggle.
- Find another model on Hugging Face and compare it.
- Modify the prompt to fit your selected dataset.

In [7]:
!pip install -q sentence-transformers
!pip install -q xformers
!pip install -q chromadb

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.1.1+cu121 requires torch==2.1.1, but you have torch 2.5.0 which is incompatible.
torchvision 0.16.1+cu121 requires torch==2.1.1, but you have torch 2.5.0 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradient 2.0.6 requires attrs<=19, but you have attrs 23.1.0 which is incompatible.
gradient 2.0.6 requires PyYAML==5.*, but you have pyyaml 6.0.2 which is incompatible.


In [1]:
import numpy as np 
import pandas as pd

## Load the Dataset
As you can see the notebook is ready to work with three different Datasets. Just uncomment the lines of the Dataset you want to use. 

I selected Datasets with News. Two of them have just a brief decription of the news, but the other contains the full text. 

As we are working in a free and limited space, I limited the number of news to use with the variable MAX_NEWS. Feel free to pull more if you have memory available. 

The name of the field containing the text of the new is stored in the variable *DOCUMENT* and the metadata in *TOPIC*

In [5]:
# news = pd.read_csv('/kaggle/input/topic-labeled-news-dataset/labelled_newscatcher_dataset.csv', sep=';')
# MAX_NEWS = 1000
# DOCUMENT="title"
# TOPIC="topic"

news = pd.read_csv('bbc_news.csv',delimiter="\t")
MAX_NEWS = 1000
DOCUMENT="content"
TOPIC="title"

#news = pd.read_csv('/kaggle/input/mit-ai-news-published-till-2023/articles.csv')
#MAX_NEWS = 100
#DOCUMENT="Article Body"
#TOPIC="Article Header"

#news = "PICK A DATASET" #Ideally pick one from the commented ones above

ChromaDB requires that the data has a unique identifier. We can make it with this statement, which will create a new column called **Id**.


In [6]:
news["id"] = news.index
news.head()

,category,filename,title,content,id
0,business,001.txt,Ad sales boost Time Warner profit,Quarterly profits at US media giant TimeWarne...,0
1,business,002.txt,Dollar gains on Greenspan speech,The dollar has hit its highest level against ...,1
2,business,003.txt,Yukos unit buyer faces loan claim,The owners of embattled Russian oil giant Yuk...,2
3,business,004.txt,High fuel prices hit BA's profits,British Airways has blamed high fuel prices f...,3
4,business,005.txt,Pernod takeover talk lifts Domecq,Shares in UK drinks and food firm Allied Dome...,4


In [7]:
#Because it is just a course we select a small portion of News.
subset_news = news.head(MAX_NEWS)

## Import and configure the Vector Database
I'm going to use ChromaDB, the most popular OpenSource embedding Database. 

First we need to import ChromaDB, and after that import the **Settings** class from **chromadb.config** module. This class allows us to change the setting for the ChromaDB system, and customize its behavior. 

In [8]:
import chromadb
from chromadb.config import Settings

Now we need to create the seetings object calling the **Settings** function imported previously. We store the object in the variable **settings_chroma**.

Is necessary to inform two parameters 
* chroma_db_impl. Here we specify the database implementation and the format how store the data. I choose ***duckdb***, because his high-performace. It operate primarly in memory. And is fully compatible with SQL. The store format ***parquet*** is good for tabular data. With good compression rates and performance. 

* persist_directory: It just contains the directory where the data will be stored. Is possible work without a directory and the data will be stored in memory without persistece, but Kaggle dosn't support that. 

In [12]:
chroma_client = chromadb.PersistentClient(path="data/")

## Filling and Querying the ChromaDB Database
The Data in ChromaDB is stored in collections. If the collection exist we need to delete it. 

In the next lines, we are creating the collection by calling the ***create_collection*** function in the ***chroma_client*** created above.

In [13]:
collection_name = "news_collection"
if len(chroma_client.list_collections()) > 0 and collection_name in [chroma_client.list_collections()[0].name]:
        chroma_client.delete_collection(name=collection_name)

collection = chroma_client.create_collection(name=collection_name)

It's time to add the data to the collection. Using the function ***add*** we need to inform, at least ***documents***, ***metadatas*** and ***ids***. 
* In the **document** we store the big text, it's a different column in each Dataset. 
* In **metadatas**, we can informa a list of topics. 
* In **id** we need to inform an unique identificator for each row. It MUST be unique! I'm creating the ID using the range of MAX_NEWS. 


In [14]:

collection.add(
    documents=subset_news[DOCUMENT].tolist(),
    metadatas=[{TOPIC: topic} for topic in subset_news[TOPIC].tolist()],
    ids=[f"id{x}" for x in range(MAX_NEWS)],
)

In [15]:
results = collection.query(query_texts=["laptop"], n_results=10 )

print(results)

{'ids': [['id210', 'id46', 'id334', 'id55', 'id116', 'id406', 'id129', 'id68', 'id185', 'id338']], 'embeddings': None, 'documents': [[" Consumer electronics giants Hitachi and Matshushita Electric are joining forces to share and develop technology for flat screen televisions.  The tie-up comes as the world's top producers are having to contend with falling prices and intense competition. The two Japanese companies will collaborate in research & development, production, marketing and licensing. They said the agreement would enable the two companies to expand the plasma display TV market globally.  Plasma display panels are used for large, thin TVs which are replacing old-style televisions. The display market for high-definition televisions is split between models using plasma display panels and others - manufactured by the likes of Sony and Samsung - using liquid-crystal displays (LCDs). The deal will enable Hitachi and Matsushita, which makes Panasonic brand products, to develop new te

## Vector MAP

In [16]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

ModuleNotFoundError: No module named 'matplotlib'

In [17]:

getado = collection.get(ids="id141", 
                       include=["documents", "embeddings"])

In [18]:
word_vectors = getado["embeddings"]
word_list = getado["documents"]
word_vectors

array([[ 6.99756294e-03, -5.59732085e-04, -1.15083801e-02,
         5.36816940e-02,  7.77340084e-02, -7.26560503e-02,
         8.80583469e-03,  9.31546465e-02, -1.39057320e-02,
        -1.00719491e-02, -3.90935168e-02,  1.94387790e-02,
        -2.40104701e-02,  2.34180801e-02,  3.50640789e-02,
        -4.10372801e-02, -7.45410752e-03,  1.38830636e-02,
        -4.51422483e-02,  4.01724093e-02, -2.87932418e-02,
        -6.10837527e-02, -1.16567407e-02, -3.03301327e-02,
         6.08684123e-03,  2.71561909e-02,  2.01890226e-02,
        -3.62420902e-02, -2.97483038e-02,  2.49429643e-02,
        -8.81100819e-02, -2.53466330e-02, -2.89995484e-02,
        -8.29512440e-03,  3.39545906e-02,  1.67706255e-02,
         3.71570252e-02,  5.87038957e-02,  2.40463503e-02,
        -4.94027063e-02, -3.56398374e-02, -4.17391062e-02,
         1.09790370e-01, -1.33552149e-01,  2.47979034e-02,
         1.40881594e-02, -1.44505529e-02,  2.75919903e-02,
        -2.44297925e-02, -1.62558246e-03,  4.79334705e-0

Once we have our information inside the Database we can query It, and ask for data that matches our needs. The search is done inside the content of the document, and it dosn't look for the exact word, or phrase. The results will be based on the similarity between the search terms and the content of documents. 

The metadata is not used in the search, but they can be utilized for filtering or refining the results after the initial search. 


## Loading the model and creating the prompt
TRANSFORMERS!!
Time to use the library **transformers**, the most famous library from [hugging face](https://huggingface.co/) for working with language models. 

We are importing: 
* **Autotokenizer**: It is a utility class for tokenizing text inputs that are compatible with various pre-trained language models.
* **AutoModelForCasualLLM**: it provides an interface to pre-trained language models specifically designed for language generation tasks using causal language modeling (e.g., GPT models), or the model used in this notebook ***databricks/dolly-v2-3b***.
* **pipeline**: provides a simple interface for performing various natural language processing (NLP) tasks, such as text generation (our case) or text classification. 

The model selected is [dolly-v2-3b](https://huggingface.co/databricks/dolly-v2-3b), the smallest Dolly model. It have 3billion paramaters, more than enough for our sample, and works much better than GPT2. 

Please, feel free to test [different Models](https://huggingface.co/models?pipeline_tag=text-generation&sort=trending), you need to search for NLP models trained for text-generation. My recomendation is choose "small" models, or we will run out of memory in kaggle.  


In [19]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

model_id = "databricks/dolly-v2-3b"
tokenizer = AutoTokenizer.from_pretrained(model_id)
lm_model = AutoModelForCausalLM.from_pretrained(model_id)

/Users/weemoe/Documents/Ironhack/Week 7/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


The next step is to initialize the pipeline using the objects created above. 

The model's response is limited to 256 tokens, for this project I'm not interested in a longer response, but it can easily be extended to whatever length you want.

Setting ***device_map*** to ***auto*** we are instructing the model to automaticaly select the most appropiate device: CPU or GPU for processing the text generation.  

In [21]:
pipe = pipeline(
    "text-generation",
    model=lm_model,
    tokenizer=tokenizer,
    max_new_tokens=256,
    device=0
)

## Creating the extended prompt
To create the prompt we use the result from query the Vector Database  and the sentence introduced by the user. 

The prompt have two parts, the **relevant context** that is the information recovered from the database and the **user's question**. 

We only need to join the two parts together to create the prompt that we are going to send to the model. 

You can limit the lenght of the context passed to the model, because we can get some Memory problems with one of the datasets that contains a realy large text in the document part. 

In [22]:
question = "Can I buy a Toshiba laptop?"
context = " ".join([f"#{str(i)}" for i in results["documents"][0]])
#context = context[0:5120]
prompt_template = f"Relevant context: {context}\n\n The user's question: {question}"
prompt_template

'Relevant context: # Consumer electronics giants Hitachi and Matshushita Electric are joining forces to share and develop technology for flat screen televisions.  The tie-up comes as the world\'s top producers are having to contend with falling prices and intense competition. The two Japanese companies will collaborate in research & development, production, marketing and licensing. They said the agreement would enable the two companies to expand the plasma display TV market globally.  Plasma display panels are used for large, thin TVs which are replacing old-style televisions. The display market for high-definition televisions is split between models using plasma display panels and others - manufactured by the likes of Sony and Samsung - using liquid-crystal displays (LCDs). The deal will enable Hitachi and Matsushita, which makes Panasonic brand products, to develop new technology and improve their competitiveness. Hitachi recently announced a deal to buy plasma display technology fro

Now all that remains is to send the prompt to the model and wait for its response!


In [ ]:
lm_response = pipe(prompt_template)
print(lm_response[0]["generated_text"])

This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (2048). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.
